# TV Script Generation
In this project, we generate our own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs. We'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network we'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
We'll be using a subset of the original dataset. It consists of only the scenes in Moe's Tavern. This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [5]:
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [6]:
view_sentence_range = (0, 10)

import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.251908396946565
Number of lines: 4258
Average number of words in each line: 11.50164396430249

The sentences 0 to 10:

Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.



## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing. We implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, we first need to transform the words to ids.  In this function, we create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

The function returns these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [7]:
import numpy as np
import problem_unittests as tests

from collections import Counter

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    word_counts = Counter(text)
    sorted_words = sorted(word_counts, key=word_counts.get, reverse=True)
    vocab_to_int = dict()
    int_to_vocab = dict()

    for i, word in enumerate(sorted_words):
        vocab_to_int[word] = i
        int_to_vocab[i] = word
    return vocab_to_int, int_to_vocab

tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

We implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||". In this  dictionary we include the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it. This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. It's important we don't use a token that could be confused as a word (i.e., instead of using the token "dash", here we use "||dash||").

In [8]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    tokens = {'.': '||Period||',
              ',': '||Comma||', 
              '"': '||Quotation_Mark||',
              ';': '||Semicolon||',
              '!': '||Exclamation_Mark||',
              '?': '||Question_Mark',
              '(': '||Left_Parenthesis||',
              ')': '||Right_Parenthesis||',
              '--': '||Dash||',
              '\n': '||Return||'
             }
    return tokens

tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [9]:
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is our first checkpoint that saves the preprocessed data to disk.

In [10]:
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
Here are the function we need to implement in order to build a RNN:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [11]:
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.3'), 'Please use TensorFlow version 1.3 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.9.0
Default GPU Device: /device:GPU:0


### Input
The `get_inputs()` function creates the following TF Placeholders for the Neural Network :
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

It returns the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [12]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    input = tf.placeholder(tf.int32, shape=(None, None), name='input')
    targets = tf.placeholder(tf.int32, shape=(None, None), name='targets')
    learning_rate = tf.placeholder(tf.float32, shape=None, name='learning_rate')
    return input, targets, learning_rate

tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [13]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    n_layers = 1
    def lstm_cell(size):
        lstm = tf.contrib.rnn.BasicLSTMCell(size)
        return lstm
    
    Cell = tf.contrib.rnn.MultiRNNCell([lstm_cell(rnn_size) for _ in range(n_layers)])
    InitialState = tf.identity(Cell.zero_state(batch_size, tf.float32), name='initial_state')
    return Cell, InitialState

tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Here we apply embedding to `input_data` using TensorFlow and return the embedded sequence.

In [14]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    embedding = tf.Variable(tf.random_uniform((vocab_size, embed_dim), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, input_data)
    return embed

tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
We've created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

The function returns the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [15]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    Outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    FinalState = tf.identity(final_state, name="final_state")
    return Outputs, FinalState

tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
We apply the functions implemented above to:
- Apply embedding to `input_data` using our `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [16]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    embed = get_embed(input_data, vocab_size, embed_dim)
    Outputs, FinalState = build_rnn(cell, embed)
    Logits = tf.contrib.layers.fully_connected(Outputs, vocab_size, activation_fn=None)
    return Logits, FinalState

tests.test_build_nn(build_nn)

Tests Passed


### Batches
The function `get_batches` creates batches of input and targets using `int_text`.  Each batch is a Numpy array with the shape `(number of batches, 2, batch size, sequence length)` and contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If there is not enough data to fill the last batch, it is dropped.

For example, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` returns a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [18]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    words_per_batch = batch_size * seq_length
    n_batches = len(int_text)//words_per_batch
    
    # Create an empty array for storing batches
    batches = np.zeros((n_batches, 2, batch_size, seq_length))
    
    # Reshape input data
    arr = np.array(int_text)[:n_batches * words_per_batch] 
    arr = arr.reshape((batch_size, -1)) 
    new_arr = np.zeros((batch_size, arr.shape[1]+1)) 
    new_arr[:, :arr.shape[1]] = arr
    new_arr[:, arr.shape[1]] = np.append(arr[1:, 0], arr[0, 0]) 
    
    # Fill the array
    for n in range(0, arr.shape[1], seq_length):       
        batch_idx = int(n/seq_length)
        batches[batch_idx, 0] = new_arr[:, n:n+seq_length] # x
        batches[batch_idx, 1] = new_arr[:, n+1:n+seq_length+1] # y
    return batches

tests.test_get_batches(get_batches)

Tests Passed


In [19]:
# For debugging 
print(get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2) )

[[[[ 1.  2.]
   [ 7.  8.]
   [13. 14.]]

  [[ 2.  3.]
   [ 8.  9.]
   [14. 15.]]]


 [[[ 3.  4.]
   [ 9. 10.]
   [15. 16.]]

  [[ 4.  5.]
   [10. 11.]
   [16. 17.]]]


 [[[ 5.  6.]
   [11. 12.]
   [17. 18.]]

  [[ 6.  7.]
   [12. 13.]
   [18.  1.]]]]


## Neural Network Training
### Hyperparameters
Now we can tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [26]:
# Number of Epochs
num_epochs = 60

# Batch Size
batch_size = 1200

# RNN Size
rnn_size = 2048

# Embedding Dimension Size
embed_dim = 300

# Sequence Length
seq_length = 15

# Learning Rate
learning_rate = 0.025

# Show stats for every n number of batches
show_every_n_batches = 10

save_dir = './save'

### Build the Graph
Build the graph using the neural network we implemented.

In [27]:
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data. 

In [28]:
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/3   train_loss = 8.822
Epoch   3 Batch    1/3   train_loss = 7.314
Epoch   6 Batch    2/3   train_loss = 4.954
Epoch  10 Batch    0/3   train_loss = 3.534
Epoch  13 Batch    1/3   train_loss = 2.512
Epoch  16 Batch    2/3   train_loss = 1.938
Epoch  20 Batch    0/3   train_loss = 1.305
Epoch  23 Batch    1/3   train_loss = 0.932
Epoch  26 Batch    2/3   train_loss = 0.621
Epoch  30 Batch    0/3   train_loss = 0.413
Epoch  33 Batch    1/3   train_loss = 0.301
Epoch  36 Batch    2/3   train_loss = 0.258
Epoch  40 Batch    0/3   train_loss = 0.236
Epoch  43 Batch    1/3   train_loss = 0.217
Epoch  46 Batch    2/3   train_loss = 0.218
Epoch  50 Batch    0/3   train_loss = 0.215
Epoch  53 Batch    1/3   train_loss = 0.205
Epoch  56 Batch    2/3   train_loss = 0.207
Model Trained and Saved


## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [29]:
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [30]:
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Here we get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name) by supplying the following names as arguments:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

The function returns the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [31]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    InputTensor = loaded_graph.get_tensor_by_name("input:0")
    InitialStateTensor = loaded_graph.get_tensor_by_name("initial_state:0")
    FinalStateTensor = loaded_graph.get_tensor_by_name("final_state:0")
    ProbsTensor = loaded_graph.get_tensor_by_name("probs:0")
    return InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor

tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
The `pick_word()` function selects the next word using `probabilities`.

In [32]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    idx = np.random.choice(list(int_to_vocab.keys()), 1, p=probabilities)[0]
    next_word = int_to_vocab[idx]
    return next_word

tests.test_pick_word(pick_word)

Tests Passed


## Generate TV Script
The code below will generate the TV script for us.  The `gen_length` should be set to the desired length of the generated TV script.

In [33]:
gen_length = 400

# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[0][dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

INFO:tensorflow:Restoring parameters from ./save
moe_szyslak: oh yeah? how 'bout this scary face?(scary noises)
lisa_simpson:(explaining) homer? i spy help the only, i hate your get to, homer...
lisa_simpson: yeah, i... to you are those... i didn't.
jacques:(pointedly, to ya) a minute that's the wrong, homer?(taps to being through.
barney_gumble:(quickly) yeah, poor...
colonel:(singing)
moe_szyslak:(gasp) oh, not quite. then i was a better man this way: you're a lot off.
homer_simpson:(tough) check...
homer_simpson:(singing) hey, the guys...(third head) maggie, homer, not has on, and did it my job.
apu_nahasapeemapetilon:(to lisa) look, tough the end, stop you know what i was just 'cause.
homer_simpson:(to) oh, why...
homer_simpson:(big wide) look, wait?...
lenny_leonard:(to, in). for i have that right, i better it... in the
moe_szyslak: gee, you...
moe_szyslak: all right, homer simpson? entire goin' what, fritz!
homer_simpson:(proudly) yeah, i'd like a and a girl room?
moe_szyslak:(ge

# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckily there's more data!  As we mentioned in the beggining of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data). You are free to train your neural network on all the data.  